In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import colorsys
from PIL import Image
from collections import defaultdict
from skimage.graph import pixel_graph
import skimage
from scipy.spatial.distance import cdist
from skimage.measure import find_contours

In [40]:
cellPath = 'segmentations/cellCycle/edUOnly/cellSize/P05/72hr/aorta01/cellCycleEdUOnlyImage_P05_72hr_aorta01_cellSize6New_seg.npy'
nucPath = 'segmentations/cellCycle/edUOnly/nuclei/P05/72hr/aorta01/cellCycleEdUOnlyImage_P05_72hr_aorta01_nuclei6_seg.npy'
filterPath = 'results/cellCycle/edUOnly/thresholded/P05/72hr/aorta01/eduActive_P05_72hr_72hr_6.csv'
savePath = "results/cellCycle/edUOnly/clusteredCells/P05/72hr/aorta01/clusteredActiveCells_P05_72hr_aorta01_6Updated.csv"

In [41]:
def merge_dictionary_keys(dictionary):
    merged_dictionary = {}

    # Iterate over each key-value pair in the dictionary
    for key, value in dictionary.items():
        merged = False

        # Check if any existing merged key already contains common values with the current key
        for merged_key, merged_value in merged_dictionary.items():
            if np.intersect1d(value, merged_value).size > 0:
                # Merge the current key with the existing merged key
                merged_dictionary[merged_key] = np.union1d(value, merged_value)
                merged = True
                break

        if not merged:
            # Add the current key-value pair to the merged dictionary
            merged_dictionary[key] = value

    return merged_dictionary

In [ ]:

#load data
nucs = np.load(nucPath, allow_pickle=True).item()['masks']
dfFilter = pd.read_csv(filterPath)
cells = np.load(cellPath, allow_pickle=True).item()['masks']

#filter active nucs
nucsFilter = np.where(np.isin(nucs,dfFilter['label'].values), nucs, 0)

# get cellIDs of active nuclei
cellIds = np.array([0])
for i in np.unique(nucsFilter):
    if i != 0: #skip background
        unique , counts = np.unique(cells[nucsFilter==i], return_counts = True)
        uniqueNew = []
        for i in range(len(unique)):
            if unique[i] != 0:
                if (counts[i]/counts.sum()) >= 0.1:
                    uniqueNew.append(unique[i])
        cellIds = np.concatenate([cellIds, np.asarray(uniqueNew)])
cellIds = cellIds[np.nonzero(cellIds)]
cellsFilter = np.where(np.isin(cells,cellIds), cells, 0)

#get contours of active cells
contours = {}
for label in np.unique(cellsFilter):
    if label != 0:
        # Create a binary mask for the current cell and find its contours
        maskFilt  = np.where(cellsFilter == label, 1, 0)
        contours[label] = find_contours(maskFilt.astype(float), level=0.5)[0]

#### get clusters
clustersOld = {}
k = 1
keys = list(contours.keys()) # List of cell contour keys
for i, key1 in enumerate(keys):
    clustersOld[k] = [key1]
    for key2 in keys[i + 1:]:
        # Calculate minimum distance between contour points of two cell contours (= min distance between 2 cells)
        dist = cdist(contours[key1], contours[key2]).min()
        # If cells are less than 11 pixels apart, define as cluster
        if dist <= 11:
            value_exists = any(key1 in array for array in clustersOld.values())
            # If key1 is already in a cluster, append key2 to that cluster
            if value_exists:
                for key, value in clustersOld.items():
                    if key1 in value:
                        clustersOld[key].append(key2)
    k += 1

for key, value in clustersOld.items():
    # Convert the list value into a NumPy array
    clustersOld[key] = np.array(value)
clusters = merge_dictionary_keys(clustersOld)

#generate cluster image from EdU active cells
clustersImage = cellsFilter.copy()
for key in clusters.keys():
    for cell in clusters[key]:
        clustersImage = np.where(clustersImage == cell, key, clustersImage)

#label clusters in df
dfFilter['cluster'] = 0
k = 1
for cluster in np.unique(clustersImage):
    if cluster != 0:
         # Get unique nuclei corresponding to the current cluster
        unique = np.unique(nucsFilter[np.where(clustersImage == cluster)])
        uniqueWOZero = unique[np.nonzero(unique)]
        if len(uniqueWOZero) > 0:
            dfFilter.loc[dfFilter['label'].isin(uniqueWOZero), 'cluster'] = cluster

# Check for clusters touching the image border
borderingCoord = np.concatenate(([0,1,2,3,4,5], np.arange(clustersImage.shape[0] -6, clustersImage.shape[0])))
dfFilter['border'] = 0
for cl in np.unique(clustersImage):
    if cl != 0:
        # Check if any part of the cluster touches the defined border coordinates
        if np.any(np.isin(np.where(clustersImage == cl), borderingCoord)) == True:
            dfFilter.loc[dfFilter['cluster'] == cl, 'border'] = 1

# get graph of cell boundaries to check which cells are touching each other
label_mask = cells # load your image here
g, nodes = pixel_graph(
    label_mask,
    mask=label_mask.astype(bool),
    connectivity=2,  # count diagonals in 2D
)
g.eliminate_zeros() # Remove zero weights from the graph
coo = g.tocoo()
center_coords = nodes[coo.row]
neighbor_coords = nodes[coo.col]
center_values = label_mask.ravel()[center_coords]
neighbor_values = label_mask.ravel()[neighbor_coords]
pairs = defaultdict(list)
for i, j in zip(center_values, neighbor_values):
    pairs[i].append(j)
# Check neighboring cells of clusters and if they are touching the image border
dfFilter['borderNeighbour'] = 0
borderingNucs = np.concatenate(([0,1,2], np.arange(clustersImage.shape[0] -3, clustersImage.shape[0])))
for cl in np.unique(clustersImage):
    if cl != 0:
        unique = np.unique(cells[np.where(clustersImage == cl)])
        uniqueWOZero = unique[np.nonzero(unique)]
        for i in uniqueWOZero:
            touching = np.unique(pairs[i])
            for t in touching:
                if t != cl:
                    #check which cells are touching border
                    if np.any(np.isin(np.where(cells == t), borderingCoord)) == True:
                        #check for which cells I don't see the nucleus full
                        uniqueNucs = np.unique(nucs[cells == t])
                        uniqueNucs = uniqueNucs[np.nonzero(uniqueNucs)]
                        for nuc in uniqueNucs:
                            if np.any(np.isin(np.where(nucs == nuc), borderingNucs)) == True:
                                dfFilter.loc[dfFilter['cluster'] == cl, 'borderNeighbour'] = 1

In [214]:
dfFilter.to_csv(savePath, index=False)